In [ ]:
# https://www.youtube.com/watch?v=UbvkhuqVqUI

In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

/Users/My/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/My/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/My/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/My/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework

In [4]:
# prepare independent and dependent features
def prepare_data(data, num_steps):
    X = []
    y = []
    for i in range(len(data) - num_steps):
        # if num_steps=3, then for each i
        # X = [data[i], data[i+1], data[i+2]] and y = data[i+3]
        # e.g. at i = 0, X = [110, 125, 133] and y = 146
        end = i + num_steps
        X.append(data[i:end])
        y.append(data[end])
    return np.array(X), np.array(y)

In [5]:
# define input sequence
data = [110, 125, 133, 146, 158, 172, 187, 196, 210]
# set the number of timesteps
num_steps = 3
# split into samples
X, y = prepare_data(data, num_steps)
print(X), print(y)

[[110 125 133]
 [125 133 146]
 [133 146 158]
 [146 158 172]
 [158 172 187]
 [172 187 196]]
[146 158 172 187 196 210]


(None, None)

In [6]:
X.shape

(6, 3)

In [7]:
# reshape X from 2d into 3d
num_features = 1
X = X.reshape((X.shape[0], X.shape[1], num_features))

In [8]:
# Build the LSTM Model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(num_steps, num_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=0)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
# prediction for the next 10 days
input = np.array([187, 196, 210])
output = []

for i in range(10):
    # collect the last 3 data points
    tmp_in = input[-num_steps:]
    print("{} day input {}".format(i, tmp_in))
    # reshape it for use by model
    tmp_in = tmp_in.reshape((1, num_steps, num_features))
    # predict the 4th data point
    tmp_out = model.predict(tmp_in, verbose=0)
    print("{} day output {}".format(i, tmp_out))
    # save the predicated data point in input
    # this will become part of the next 3 data points in the next iteration
    input = np.append(input, tmp_out[0][0])
    # save the predicated data point in output
    output.append(tmp_out[0][0])

print(output)

0 day input [187 196 210]
0 day output [[230.55342]]
1 day input [196.         210.         230.55342102]
1 day output [[244.82365]]
2 day input [210.         230.55342102 244.82365417]
2 day output [[263.9413]]
3 day input [230.55342102 244.82365417 263.9413147 ]
3 day output [[285.99326]]
4 day input [244.82365417 263.9413147  285.99325562]
4 day output [[305.9175]]
5 day input [263.9413147  285.99325562 305.91751099]
5 day output [[329.91086]]
6 day input [285.99325562 305.91751099 329.91085815]
6 day output [[355.7526]]
7 day input [305.91751099 329.91085815 355.75259399]
7 day output [[381.87625]]
8 day input [329.91085815 355.75259399 381.87625122]
8 day output [[411.50314]]
9 day input [355.75259399 381.87625122 411.50314331]
9 day output [[443.02194]]
[230.55342, 244.82365, 263.9413, 285.99326, 305.9175, 329.91086, 355.7526, 381.87625, 411.50314, 443.02194]


In [10]:
data

[110, 125, 133, 146, 158, 172, 187, 196, 210]

In [11]:
output

[230.55342,
 244.82365,
 263.9413,
 285.99326,
 305.9175,
 329.91086,
 355.7526,
 381.87625,
 411.50314,
 443.02194]

In [ ]:
# visualize the output
import matplotlib.pyplot as plt
# this causes Jupyter to crash; possibly because it's plt.plot is incompatible with np.arange
plt.plot(np.arange(1, 10), data)
plt.plot(np.arange(10, 20), output)